# SQL 的五十道練習：初學者友善的資料庫入門

> 垂直與水平合併資料

In [1]:
%LOAD sqlite3 db=../databases/imdb.db timeout=2 shared_cache=true

## 複習一下

在第十章「子查詢」我們提過常見的子查詢結構有三種外型：

1. 接續在 `WHERE` 保留字後的結構外型。
2. 接續在 `SELECT` 保留字後的結構外型。
3. 接續在 `FROM` 保留字後的結構外型，這裡要注意的是先前別名是針對欄位名稱，這裡則是將先執行的 SQL 敘述查詢結果視為像資料表（實際上並不是）的存在。

舉例來說，我們想知道哪幾部電影 Tom Hanks 有演出？要回答這個問題首先得檢視 `imdb` 學習資料庫中具有的三個資料表：`actors`、`casting` 與 `movies`。

In [2]:
SELECT *
  FROM actors
 LIMIT 5;

id,name
1,Aamir Khan
2,Aaron Eckhart
3,Aaron Lazar
4,Abbas-Ali Roomandi
5,Abbey Lee


In [3]:
SELECT *
  FROM casting
 LIMIT 5;

movie_id,actor_id,ord
1,2944,1
1,2192,2
1,330,3
1,3134,4
1,552,5


In [4]:
SELECT *
  FROM movies
 LIMIT 5;

id,title,release_year,rating,director,runtime
1,The Shawshank Redemption,1994,9.3,Frank Darabont,142
2,The Godfather,1972,9.2,Francis Ford Coppola,175
3,The Dark Knight,2008,9,Christopher Nolan,152
4,The Godfather Part II,1974,9,Francis Ford Coppola,202
5,12 Angry Men,1957,9,Sidney Lumet,96


回到問題本身「我們想知道哪幾部電影 Tom Hanks 有演出？」我們必須先知道 Tom Hanks 的演員編號 `actor_id` 是多少，這樣我們才能夠從 `casting` 資料表對應出哪些電影編號 `movie_id`，進而再由 `movies` 資料表取得電影資訊 `title`。假定 Tom Hanks 有演出的電影編號為 `x`，我們可以寫出以下的 SQL 敘述得到這個問題的答案：

```sql
SELECT title
  FROM movies
 WHERE id IN x;
```

但是 `x` 必須要先經過一個 SQL 敘述查詢才能得知為多少，假定 Tom Hanks 的演員編號為 `y`，獲得關鍵 `x` 的 SQL 敘述為：

```sql
SELECT movie_id
  FROM casting
 WHERE actor_id = y;
```

`y` 也必須要再一個 SQL 敘述查詢才能得知為多少，獲得關鍵 `y` 的 SQL 敘述為：

```sql
SELECT id
  FROM actors
 WHERE name = 'Tom Hanks';
```

接著我們可以將 `x` 與 `y` 依序替換為 SQL 敘述，並用小括號 `()` 包裝起來，就能成功將本來應該分三次、先後執行的 SQL 敘述，調整為子查詢的結構外型。

In [5]:
SELECT title
  FROM movies
 WHERE id IN (
           SELECT movie_id
             FROM casting
            WHERE actor_id = (
                                 SELECT id
                                   FROM actors
                                  WHERE name = 'Tom Hanks'
                             )
       );

title
Forrest Gump
Saving Private Ryan
The Green Mile
Toy Story
Toy Story 3
Catch Me If You Can


回答「我們想知道哪幾部電影 Tom Hanks 有演出？」問題的過程展現了透過子查詢，我們已經有能力從查詢「單個」資料表擴展至查詢「多個」資料表，而除了子查詢以外，垂直與水平合併也是整合多個資料表內容的技巧。

## 關聯

在第一章「簡介」我們提過關聯式資料庫是以列（Rows）與欄（Columns）所組成的二維表格形式記錄，並且遵守關聯式模型準則設計，經由設計資料表彼此之間的關聯，讓資料的重複性降低，除了提高儲存效率，亦有便於維護的優點。而所謂的關聯，具體來說就是讓資料從兩個維度合併：

1. 垂直合併：從垂直的方向關聯資料的列（觀測值）。
2. 水平合併：從水平的方向關聯資料的欄（變數）。

更簡單的說明，垂直合併就是列的結合，`(m, n)` 外型與 `(m, n)` 外型的資料垂直合併後為 `(2m, n)` 外型；水平合併就是欄的結合，`(m, n)` 外型與 `(m, n)` 外型的資料水平合併後為 `(m, 2n)` 外型。

## 以 `UNION` 垂直合併

以 `UNION` 保留字合併 SQL 敘述，資料合併依據是 `SELECT` 保留字後的順序。

```sql
A SQL statement
UNION
Another SQL statement
```

舉例來說，第一個 SQL 敘述從 `movies` 資料表中篩選出兩位導演。

In [6]:
SELECT director
  FROM movies
 WHERE director IN ('Christopher Nolan', 'Steven Spielberg');

director
Christopher Nolan
Steven Spielberg
Christopher Nolan
Steven Spielberg
Christopher Nolan
Christopher Nolan
Christopher Nolan
Steven Spielberg
Christopher Nolan
Steven Spielberg


第二個 SQL 敘述從 `actors` 資料表篩選出兩位演員。

In [7]:
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Leonardo DiCaprio');

name
Leonardo DiCaprio
Tom Hanks


在兩個 SQL 敘述中以 `UNION` 保留字合併原本記錄於不同資料表的兩位導演與兩位演員。

In [8]:
SELECT director AS my_favorites
  FROM movies
 WHERE director IN ('Christopher Nolan', 'Steven Spielberg')
 UNION
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Leonardo DiCaprio');

my_favorites
Christopher Nolan
Leonardo DiCaprio
Steven Spielberg
Tom Hanks


從前述這個垂直合併的查詢結果中，我們可以觀察到重複的觀測值會被省略並且有遞增排序，就像是加入了 `GROUP BY` 保留字一般的效果。假如不希望重複的觀測值被剔除、也不想要有排序，可以改用 `UNION ALL` 保留字垂直合併。

In [9]:
SELECT director AS my_favorites
  FROM movies
 WHERE director IN ('Christopher Nolan', 'Steven Spielberg')
 UNION ALL
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Leonardo DiCaprio');

my_favorites
Christopher Nolan
Steven Spielberg
Christopher Nolan
Steven Spielberg
Christopher Nolan
Christopher Nolan
Christopher Nolan
Steven Spielberg
Christopher Nolan
Steven Spielberg


### 值得注意的垂直合併特性

上、下 SQL 敘述所選擇的欄位數要相同，不然會發生錯誤。

```sql
SELECT director AS my_favorites
  FROM movies
 WHERE director IN ('Christopher Nolan', 'Steven Spielberg')
 UNION
SELECT name,
       id  -- do not have the same number of result columns
  FROM actors
 WHERE name IN ('Tom Hanks', 'Leonardo DiCaprio');
```

```
Error while executing SQL query on database 'imdb': SELECTs to the left and right of UNION do not have the same number of result columns
```

若有使用到 `ORDER BY` 保留字要放在 `UNION` 之後，不然會發生錯誤。

```sql
SELECT director AS my_favorites
  FROM movies
 WHERE director IN ('Christopher Nolan', 'Steven Spielberg')
 ORDER BY my_favorites -- ORDER BY clause should come after UNION not before
 UNION
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Leonardo DiCaprio');
```

```
Error while executing SQL query on database 'imdb': ORDER BY clause should come after UNION not before
```

## 以 `JOIN` 水平合併

有別於垂直合併的依據是 `SELECT` 保留字後的順序，水平合併的依據是資料表之間的連接鍵（Join keys），也就是用來關聯兩張資料表的欄位。由於資料是以水平維度進行合併，因此還會有左資料表與右資料表之分。

```sql
SELECT left_table.columns,
       right_table.columns
  FROM left_table
  JOIN right_table
    ON left_table.join_key = right_table.join_key;
```

在 `FROM` 保留字之後的資料表被稱為左資料表（Left table）或稱主要資料表；在 `JOIN` 保留字之後的資料表被稱為右資料表（Right table）或稱次要資料表。如果要水平合併的資料表不只兩個，可以再添加 `JOIN` 與 `ON` 納入更多的次要資料表。

```sql
SELECT left_table.columns,
       right_table_one.columns,
       right_table_two.columns
  FROM left_table
  JOIN right_table_one
    ON left_table.join_key = right_table_one.join_key
  JOIN right_table_two
    ON left_table.join_key|right_table_one.join_key = right_table_two.join_key;
```

在 `ON` 保留字之後的敘述包含了資料表名稱與連接鍵的宣告 `table.join_key` 中間用 `.` 來區分資料表名稱與連接鍵。而在 `SELECT` 保留字之後所選擇的欄位，因為能夠從主要資料表與次要資料表中選擇，為了提升可讀性、同時也避免模糊錯誤（Ambiguity error）的發生，也同樣會以 `table.columns` 明確表示資料表名稱與所選擇欄位。

我們再一次回到問題「我們想知道哪幾部電影 Tom Hanks 有演出？」我們必須先知道 Tom Hanks 的演員編號 `actor_id` 是多少，這樣我們才能夠從 `casting` 資料表對應出哪些電影編號 `movie_id`，進而再由 `movies` 資料表取得電影資訊 `title`。除了採用本章開頭的子查詢解題，我們也能運用水平合併將三個資料表連接後解題，而資料表之間的關係，能夠透過實體關係圖（ER Diagram, Entity Relationship Diagram）來呈現。

實體關係圖會將關聯式資料庫中每個資料表像清單般展開，最上方是該資料表名稱，置頂且粗體的欄位名稱則標註了該資料表中用來區隔「不重複」觀測值的變數，也就是所謂的主鍵（Primary key）；資料表與資料表之間的連線則描述兩者能夠透過連接鍵關聯。舉例來說，從 `imdb` 資料庫的實體關係圖，可以得知 `actors` 的主鍵為 `id`、可以透過 `actors.id` 與 `casting.actor_id` 連接；`movies` 主鍵為 `id`、可以透過 `movies.id` 與 `casting.movie_id` 連接。因此回答問題「我們想知道哪幾部電影 Tom Hanks 有演出？」可以將三張資料表連接後，篩選 Tom Hanks 的演員姓名，最後選擇電影名稱。

![](../images/imdb-erd.png)

In [10]:
SELECT movies.title
  FROM movies
  JOIN casting
    ON movies.id = casting.movie_id
  JOIN actors
    ON casting.actor_id = actors.id
 WHERE actors.name = 'Tom Hanks';

title
Forrest Gump
Saving Private Ryan
The Green Mile
Toy Story
Toy Story 3
Catch Me If You Can


如果沒有選擇指定的欄位，`SELECT *` 可以觀察到水平合併的效果，是將三個資料表的欄都集中到了一個查詢結果中，`id:runtime` 來自 `movies` 資料表；`movie_id:ord` 來自 `casting` 資料表；`id:name` 來自 `actors` 資料表。

In [11]:
SELECT *
  FROM movies
  JOIN casting
    ON movies.id = casting.movie_id
  JOIN actors
    ON casting.actor_id = actors.id
 WHERE actors.name = 'Tom Hanks';

id,title,release_year,rating,director,runtime,movie_id,actor_id,ord,id,name
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,2957,1,2957,Tom Hanks
24,Saving Private Ryan,1998,8.6,Steven Spielberg,169,24,2957,1,2957,Tom Hanks
26,The Green Mile,1999,8.6,Frank Darabont,189,26,2957,1,2957,Tom Hanks
75,Toy Story,1995,8.3,John Lasseter,81,75,2957,1,2957,Tom Hanks
84,Toy Story 3,2010,8.3,Lee Unkrich,103,84,2957,1,2957,Tom Hanks
177,Catch Me If You Can,2002,8.1,Steven Spielberg,141,177,2957,2,2957,Tom Hanks


舉例來說，從 `covid19` 資料庫的實體關係圖，可以得知 `daily_report` 的主鍵為 `Combined_Key`、可以透過 `daily_report.Combined_Key` 與 `lookup_table.Combined_Key` 連接；`lookup_table` 主鍵為 `UID`、可以透過 `lookup_table.Country_Region` 與 `time_series.Country_Region` 連接。

![](../images/covid19-erd.png)

舉例來說，從 `nba` 資料庫的實體關係圖，可以得知 `career_summaries` 的主鍵為 `personId`、可以透過 `career_summaries.personId` 與 `players.personId` 連接；`teams` 主鍵為 `teamId`、可以透過 `teams.teamId` 與 `players.teamId` 連接。

![](../images/nba-erd.png)

舉例來說，從 `twElection2020` 資料庫的實體關係圖，可以得知 `admin_regions` 的主鍵為 `id`、可以透過 `admin_regions.id` 與 `legislative_at_large.admin_region_id`、`legislative_regional.admin_region_id`、`presidential.admin_region_id` 連接；`candidates` 主鍵為 `id`、可以透過 `candidates.id` 與 `presidential.candidate_id`、`legislative_regional.candidate_id` 連接，透過 `candidates.party_id` 與 `parties.id` 連接；`parties` 主鍵為 `id`、可以透過 `parties.id` 與 `legislative_at_large.party_id`、`candidates.party_id` 連接。

![](../images/twElection2020-erd.png)

### 值得注意的水平合併特性

留意資料的映射關係，像是一對一、一對多、多對一以及多對多的關係，舉例來說，從 `imdb` 資料庫中我們可以發現電影（`movies`）與演員（`actors`）是多對多的關係，在一部電影中會有多位演員、一位演員也能出演多部電影，因此連接一部電影與卡司時資料列數會有一對多的情況。

In [12]:
SELECT movies.title,
       casting.actor_id
  FROM movies
  JOIN casting
    ON movies.id = casting.movie_id
 WHERE movies.id = 1;

title,actor_id
The Shawshank Redemption,2944
The Shawshank Redemption,2192
The Shawshank Redemption,330
The Shawshank Redemption,3134
The Shawshank Redemption,552
The Shawshank Redemption,1086
The Shawshank Redemption,2017
The Shawshank Redemption,1384
The Shawshank Redemption,1444
The Shawshank Redemption,1813


同理，連接一位演員與卡司時資料列數也會發生一對多的情況。

In [13]:
SELECT actors.name,
       casting.movie_id
  FROM actors
  JOIN casting
    ON actors.id = casting.actor_id
 WHERE actors.name = 'Tom Hanks';

name,movie_id
Tom Hanks,11
Tom Hanks,24
Tom Hanks,26
Tom Hanks,75
Tom Hanks,84
Tom Hanks,177


養成為在欄位名稱前註明清楚資料表名稱的習慣 `table.columns`，如此一來可以提升可讀性、同時也避免模糊錯誤的發生，舉例來說，在 `movies` 與 `actors` 兩個資料表中都有欄位 `id`，`movies.id` 指的是電影編號、`actors.id` 指的是演員編號。如果沒有註明資料表名稱，會發生錯誤。

```sql
SELECT title
  FROM movies
  JOIN casting
    ON id = movie_id
  JOIN actors
    ON actor_id = id
 WHERE name = 'Tom Hanks';
```

```
Error while executing SQL query on database 'imdb': ambiguous column name: id
```

除了資料表也能夠使用子查詢作為合併資料來源，舉例來說，我們可以水平合併僅有 Tom Hanks 的 `actors` 資料表，這時記得給予子查詢的結果一個別名，當作是資料表一般（實際上不是）。

In [14]:
SELECT *
  FROM casting
  JOIN
       (
           SELECT *
             FROM actors
            WHERE name = 'Tom Hanks'
       )
       AS actor_tom_hanks
    ON casting.actor_id = actor_tom_hanks.id;

movie_id,actor_id,ord,id,name
11,2957,1,2957,Tom Hanks
24,2957,1,2957,Tom Hanks
26,2957,1,2957,Tom Hanks
75,2957,1,2957,Tom Hanks
84,2957,1,2957,Tom Hanks
177,2957,2,2957,Tom Hanks


`JOIN` 是會保留左表格與右表格**交集**的觀測值，也就是所謂的內連接（Inner join）。舉例來說，如果我們利用子查詢設計左表格為阿甘正傳與搶救雷恩大兵的電影資料；設計右表格為阿甘正傳與神鬼交鋒的演員卡司。

In [15]:
SELECT *
  FROM movies
 WHERE title IN ('Forrest Gump', 'Saving Private Ryan'); -- AS left_table

id,title,release_year,rating,director,runtime
11,Forrest Gump,1994,8.8,Robert Zemeckis,142
24,Saving Private Ryan,1998,8.6,Steven Spielberg,169


In [16]:
SELECT *
  FROM casting
 WHERE movie_id IN (11, 175); -- AS right_table

movie_id,actor_id,ord
11,2957,1
11,2481,2
11,2668,3
11,2112,4
11,1176,5
11,1051,6
11,334,7
11,1579,8
11,2674,9
11,1986,10


將左表格（阿甘正傳與搶救雷恩大兵的電影資料）與右表格（阿甘正傳與神鬼交鋒的演員卡司）水平合併，結果會是左表格與右表格**交集**的觀測值，也就是阿甘正傳的電影資料、演員卡司，而搶救雷恩大兵的電影資料與神鬼交鋒的演員卡司都不會出現在查詢結果中。

In [17]:
SELECT *
  FROM (
           SELECT *
             FROM movies
            WHERE title IN ('Forrest Gump', 'Saving Private Ryan') 
       )
       AS left_table
  JOIN
       (
           SELECT *
             FROM casting
            WHERE movie_id IN (11, 175) 
       )
       AS right_table
    ON left_table.id = right_table.movie_id;

id,title,release_year,rating,director,runtime,movie_id,actor_id,ord
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,2957,1
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,2481,2
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,2668,3
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,2112,4
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,1176,5
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,1051,6
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,334,7
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,1579,8
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,2674,9
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,1986,10


與內連接（Inner join）對應的是外連接（Outer join），改使用 `LEFT JOIN` 可以變為保留左表格（主要表格）的觀測值。也就是阿甘正傳與搶救雷恩大兵的電影資料、阿甘正傳的演員卡司，搶救雷恩大兵因為沒有演員卡司可以連接，所以得到空值 `NULL`。

In [18]:
SELECT *
  FROM (
           SELECT *
             FROM movies
            WHERE title IN ('Forrest Gump', 'Saving Private Ryan') 
       )
       AS left_table
  LEFT JOIN
       (
           SELECT *
             FROM casting
            WHERE movie_id IN (11, 175) 
       )
       AS right_table
    ON left_table.id = right_table.movie_id;

id,title,release_year,rating,director,runtime,movie_id,actor_id,ord
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,334,7
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,527,12
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,1051,6
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,1176,5
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,1285,11
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,1579,8
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,1590,13
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,1986,10
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,2112,4
11,Forrest Gump,1994,8.8,Robert Zemeckis,142,11,2390,14


雖然 SQLite 關聯式資料庫管理系統並沒有 `RIGHT JOIN` 可以直接使用來變為保留右表格（次要表格）的觀測值，但因為左、右是相對的，其實可以沿用 `LEFT JOIN` 但是把左表格與右表格互換，藉此達到預期的效果。也就是阿甘正傳與神鬼交鋒的演員卡司、阿甘正傳的電影資料，神鬼交鋒因為沒有電影資料可以連接，所以得到空值 `NULL`。

In [19]:
SELECT *
  FROM (
           SELECT *
             FROM casting
            WHERE movie_id IN (11, 175)  
       )
       AS left_table
  LEFT JOIN
       (
           SELECT *
             FROM movies
            WHERE title IN ('Forrest Gump', 'Saving Private Ryan')
       )
       AS right_table
    ON left_table.movie_id = right_table.id;

movie_id,actor_id,ord,id,title,release_year,rating,director,runtime
11,2957,1,11,Forrest Gump,1994,8.8,Robert Zemeckis,142
11,2481,2,11,Forrest Gump,1994,8.8,Robert Zemeckis,142
11,2668,3,11,Forrest Gump,1994,8.8,Robert Zemeckis,142
11,2112,4,11,Forrest Gump,1994,8.8,Robert Zemeckis,142
11,1176,5,11,Forrest Gump,1994,8.8,Robert Zemeckis,142
11,1051,6,11,Forrest Gump,1994,8.8,Robert Zemeckis,142
11,334,7,11,Forrest Gump,1994,8.8,Robert Zemeckis,142
11,1579,8,11,Forrest Gump,1994,8.8,Robert Zemeckis,142
11,2674,9,11,Forrest Gump,1994,8.8,Robert Zemeckis,142
11,1986,10,11,Forrest Gump,1994,8.8,Robert Zemeckis,142


同理，雖然 SQLite 關聯式資料庫管理系統並沒有 `FULL JOIN` 可以直接使用來變為保留左、右表格（主、次要表格）的觀測值，但我們可以把前述兩個 `LEFT JOIN` 的結果透過 `UNION` 垂直合併，藉此達到預期的效果。也就是阿甘正傳的電影資料與演員卡司、搶救雷恩大兵的電影資料、神鬼交鋒的演員卡司，神鬼交鋒因為沒有電影資料可以連接、搶救雷恩大兵因為沒有演員卡司可以連接，所以都得到空值 `NULL`。

In [20]:
SELECT left_table.id,
       left_table.title,
       right_table.actor_id
  FROM (
           SELECT *
             FROM movies
            WHERE title IN ('Forrest Gump', 'Saving Private Ryan') 
       )
       AS left_table
  LEFT JOIN
       (
           SELECT *
             FROM casting
            WHERE movie_id IN (11, 175) 
       )
       AS right_table
    ON left_table.id = right_table.movie_id
 UNION
SELECT right_table.id,
       right_table.title,
       left_table.actor_id
  FROM (
           SELECT *
             FROM casting
            WHERE movie_id IN (11, 175)  
       )
       AS left_table
  LEFT JOIN
       (
           SELECT *
             FROM movies
            WHERE title IN ('Forrest Gump', 'Saving Private Ryan')
       )
       AS right_table
    ON left_table.movie_id = right_table.id

id,title,actor_id
NULL,NULL,54
NULL,NULL,460
NULL,NULL,556
NULL,NULL,955
NULL,NULL,1068
NULL,NULL,1184
NULL,NULL,1224
NULL,NULL,1304
NULL,NULL,1856
NULL,NULL,1879


## 敘述的執行順序

截至目前我們已經學習完所有 SQL 資料查詢語言（Data Query Language, DQL）的保留字，寫作順序必須遵從標準 SQL 的規定，否則執行會發生錯誤，但是寫作順序和關聯式資料庫管理系統的執行順序是不同的，保留字的執行依照下列順序：

1. `FROM` 與 `JOIN`
2. `WHERE`
3. `GROUP BY`
4. `HAVING`
5. `SELECT`
6. `DISTINCT`
7. `UNION`
8. `ORDER BY`
9. `LIMIT`

暸解敘述的執行順序能夠幫助我們排除一些使用上可能發生的問題，舉例來說，在 `WHERE` 寫作的條件不能用 `SELECT` 寫作的縮寫或衍生計算欄位，因為 `WHERE` 比 `SELECT` 先執行。

## 重點統整

- 所謂的關聯，具體來說就是讓資料從兩個維度合併：
    - 垂直合併：以 `UNION` 從垂直的方向關聯資料的列（觀測值）。
    - 水平合併：以 `JOIN` 從水平的方向關聯資料的欄（變數）。
- 這個章節學起來的 SQL 保留字：
    - `UNION`
    - `UNION ALL`
    - `JOIN`
    - `LEFT JOIN`
    - `ON`
- 將截至目前所學的 SQL 保留字集中在一個敘述中，寫作順序必須遵從標準 SQL 的規定。

```sql
 SELECT DISTINCT columns AS alias,
        CASE WHEN condition_1 THEN result_1
             WHEN condition_2 THEN result_2
             ...
             ELSE result_n END AS alias
   FROM left_table
   JOIN | LEFT JOIN right_table
     ON left_table.join_key = right_table.join_key
  WHERE conditions
  GROUP BY columns
 HAVING conditions
  UNION | UNION ALL
Another SQL statement
  ORDER BY columns DESC
  LIMIT m;
```

## 練習題 35-39

練習題會涵蓋四個學習資料庫，記得要依據題目的需求，調整編輯器選單的學習資料庫，在自己電腦的 SQLiteStudio 寫出跟預期輸出相同的 SQL 敘述，寫作過程如果卡關了，可以參考附錄二「練習題參考解答」。

### 35. 從 `covid19` 資料庫查詢截至 2022-05-31 全球前十大確診人數的國家，參考下列的預期查詢結果。

註：本題不需考慮 `daily_report` 內的 `Last_Update` 時間戳記，`daily_report` 的數據有效期間就是 2022-05-31。

預期輸出：(10, 2) 的查詢結果。

```
Country_Region	total_confirmed
US	84227620
India	43160832
Brazil	31019038
France	29711870
Germany	26360953
United Kingdom	22486997
Korea, South	18119415
Russia	18063880
Italy	17421410
Turkey	15072747
```

### 36. 從 `twElection2020` 資料庫查詢中國國民黨、民主進步黨與親民黨在不分區立委與區域立委的得票率，參考下列的預期查詢結果。

註：不分區立委的投票資料記錄於資料表 `legislative_at_large`，區域立委的投票資料記錄於資料表 `legislative_regional`。

預期輸出：(6, 3) 的查詢結果。

```
party	election	votes_percentage
中國國民黨	不分區立委	0.3336
民主進步黨	不分區立委	0.3398
親民黨	不分區立委	0.0366
中國國民黨	區域立委	0.4071
民主進步黨	區域立委	0.4511
親民黨	區域立委	0.0043
```

### 37. 從 `nba` 資料庫查詢洛杉磯湖人隊（Los Angeles Lakers）球員的生涯場均得分 `ppg`，參考下列的預期查詢結果。

預期輸出：(17, 3) 的查詢結果。

```
team_name	player_name	ppg
Los Angeles Lakers	LeBron James	27.1
Los Angeles Lakers	Anthony Davis	23.8
Los Angeles Lakers	Russell Westbrook	22.8
Los Angeles Lakers	Carmelo Anthony	22.5
Los Angeles Lakers	Dwight Howard	15.7
Los Angeles Lakers	Kendrick Nunn	15
Los Angeles Lakers	Avery Bradley	11
Los Angeles Lakers	Malik Monk	10.3
Los Angeles Lakers	D.J. Augustin	9.5
Los Angeles Lakers	Talen Horton-Tucker	9.3
Los Angeles Lakers	Kent Bazemore	8.2
Los Angeles Lakers	Wayne Ellington	8
Los Angeles Lakers	Austin Reaves	7.3
Los Angeles Lakers	Stanley Johnson	6.3
Los Angeles Lakers	Mason Jones	5.4
Los Angeles Lakers	Mac McClung	4
Los Angeles Lakers	Wenyen Gabriel	3.6
```

### 38. 從 `nba` 資料庫查詢各個球隊的得分王（生涯場均得分 `ppg` 全隊最高）是誰，將查詢結果依隊伍名排序，參考下列的預期查詢結果。

預期輸出：(30, 3) 的查詢結果。

```
team	player	ppg
Atlanta Hawks	Trae Young	25.3
Boston Celtics	Jayson Tatum	20.9
Brooklyn Nets	Kevin Durant	27.2
Charlotte Hornets	LaMelo Ball	18.3
Chicago Bulls	DeMar DeRozan	20.8
Cleveland Cavaliers	Collin Sexton	20
Dallas Mavericks	Luka Doncic	26.4
Denver Nuggets	Nikola Jokic	19.7
Detroit Pistons	Cade Cunningham	17.4
Golden State Warriors	Stephen Curry	24.3
Houston Rockets	John Wall	19.1
Indiana Pacers	Buddy Hield	15.9
LA Clippers	Paul George	20.4
Los Angeles Lakers	LeBron James	27.1
Memphis Grizzlies	Ja Morant	21.2
Miami Heat	Jimmy Butler	17.7
Milwaukee Bucks	Giannis Antetokounmpo	21.8
Minnesota Timberwolves	Karl-Anthony Towns	23.2
New Orleans Pelicans	Zion Williamson	25.7
New York Knicks	Kemba Walker	19.5
Oklahoma City Thunder	Shai Gilgeous-Alexander	18.2
Orlando Magic	Franz Wagner	15.2
Philadelphia 76ers	Joel Embiid	26
Phoenix Suns	Devin Booker	23.5
Portland Trail Blazers	Damian Lillard	24.6
Sacramento Kings	De'Aaron Fox	19.1
San Antonio Spurs	Keldon Johnson	14.4
Toronto Raptors	Pascal Siakam	15.7
Utah Jazz	Donovan Mitchell	23.9
Washington Wizards	Bradley Beal	22.1
```

### 39. 從 `imdb` 資料庫中查詢 Tom Hanks 與 Leonardo DiCaprio 在 IMDb.com 最高評價的 250 部電影中演出哪些電影，依據 `casting` 資料表中的 `ord` 衍生計算欄位 `is_lead_actor` 註記是否為第一主角（`ord` 若為 1 表示為第一主角），將查詢結果依 `release_year` 排序，參考下列的預期查詢結果。

預期輸出：(12, 4) 的查詢結果。

```
release_year	title	name	is_lead_actor
1994	Forrest Gump	Tom Hanks	1
1995	Toy Story	Tom Hanks	1
1998	Saving Private Ryan	Tom Hanks	1
1999	The Green Mile	Tom Hanks	1
2002	Catch Me If You Can	Leonardo DiCaprio	1
2002	Catch Me If You Can	Tom Hanks	0
2006	The Departed	Leonardo DiCaprio	1
2010	Inception	Leonardo DiCaprio	1
2010	Toy Story 3	Tom Hanks	1
2010	Shutter Island	Leonardo DiCaprio	1
2012	Django Unchained	Leonardo DiCaprio	0
2013	The Wolf of Wall Street	Leonardo DiCaprio	1
```